### References
- [Deploy and run LLM locally](../../notes/vLLM.md)

In [7]:
from openai import OpenAI

# hugging face TGI 兼容 OpenAI API
client = OpenAI(base_url="http://localhost:8080/v1", api_key="dummy")
# vLLM 兼容 OpenAI API
# client = OpenAI(base_url="http://localhost:8000/v1", api_key="dummy")
# Ollama 兼容 OpenAI API
# client = OpenAI(base_url="http://localhost:11434/v1", api_key="dummy")

resp = client.chat.completions.create(
    # model="Qwen3-0.6B",
    model="google/gemma-3-4b-it",
    messages=[{"role": "user", "content": "介绍一下 vLLM 是什么"}]
)

print(resp.choices[0].message.content)


vLLM 是一个开源的，高性能的 LLM 推理服务框架，由卡夫卡（UC Berkeley）开发。它主要解决 LLM 部署中遇到的高吞吐量和低延迟问题，并提供了一系列先进的优化技术，使得 LLM 可以在资源有限的环境中高效运行。

**以下是 vLLM 的一些关键特性和亮点：**

* **PagedAttention:** 这是 vLLM 最核心的技术。传统 LLM 推理会为每个请求的 token 创建一个注意力块（Attention Block），收集完后才释放。这会导致大量内存的浪费，尤其是当有多个请求并发时。PagedAttention 借鉴了操作系统虚拟内存的管理思想，将注意力块划分为更小的块，并动态地分配和释放它们，避免了内存碎片化，大幅提高了内存利用率和推理效率。

* **动态批处理 (Dynamic Batching):**  vLLM 能够将多个请求合并成一个批次进行处理，从而最大化 GPU 的利用率。与静态批处理相比，动态批处理可以更灵活地调整批次大小，适应不同请求的长度和复杂度。

* **推理优化:** vLLM 采用了多种推理优化技术，例如零开销量化 (Zero-Copy Quantization)、多 GPU 挂载 (Multi-GPU Offloading) 等，进一步提升了推理速度。

* **易用性:** vLLM 提供了一个简洁友好的 API，方便用户集成到现有的应用中。它支持多种 LLM 模型，包括 Llama 2、Mistral、Vicuna 等。

* **多种部署方式:** vLLM 支持多种部署方式，包括：
    * **本地部署:** 可以在本地机器上运行，适用于实验和开发。
    * **云端部署:** 可以部署到 Kubernetes、Docker 等容器化平台上，适用于生产环境。

**vLLM 的优势：**

* **更高的吞吐量:**  vLLM 相比于传统的 llama.cpp 等推理引擎，可以实现更高的吞吐量，尤其是在多 GPU 环境下。
* **更低的延迟:**  PagedAttention 显著降低了延迟，使得 LLM 的响应速度更快。
* **更高效的内存利用:**  PagedAttention 避免了内存碎片化，大幅提高了内存利用率。
* **易于扩展:**  vLLM 的架构设计便于扩展

In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict

# 加载 .env 文件中的环境变量
# load_dotenv()
load_dotenv(dotenv_path='.vscode/.env.local', override=True)

def main():
    # model = os.getenv("LLM_MODEL_ID")
    # apiKey = os.getenv("LLM_API_KEY")
    # baseUrl = os.getenv("LLM_BASE_URL")
    # timeout = int(os.getenv("LLM_TIMEOUT", 60))
    model = "Qwen/Qwen3-0.6B"
    apiKey = "api_key"
    baseUrl = "http://localhost:8080"
    timeout = 60
    
    try:
        print(f"🧠 正在调用 {model} 模型 @ {baseUrl} ...")
        client = OpenAI(api_key=apiKey, base_url=baseUrl, timeout=timeout)
        messages = [
            {"role": "system", "content": "You are a helpful assistant that writes Python code."},
            {"role": "user", "content": "写一个快速排序算法"}
        ]

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.1,
            stream=True,
        )
        
        # 处理流式响应
        print("✅ 大语言模型响应成功:")
        collected_content = []
        for chunk in response:
            content = chunk.choices[0].delta.content or ""
            print(content, end="", flush=True)
            collected_content.append(content)
        print()  # 在流式输出结束后换行
        responseText = "".join(collected_content)
    
        if responseText:
            print("\n\n--- 完整模型响应 ---")
            print(responseText)

    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"❌ 调用LLM API时发生错误: {e}")
        return None

# --- 客户端使用示例 ---
if __name__ == '__main__':
    main()


🧠 正在调用 Qwen/Qwen3-0.6B 模型 @ http://localhost:8080 ...
❌ 调用LLM API时发生错误: Error code: 404
